# 12 · The Observer in the Mirror — Controls and Robustness

**Observational record associated with the book**  
*Hidden Structure in the Apparent Chaos of Prime Numbers — Computational experiments through the Euler mirror*  
© Alvaro Costa, 2025

This notebook is part of a canonical sequence of computational records.  
It introduces **no new hypotheses, conjectures, or interpretative models**.

Its sole purpose is to **record** the behaviour of arithmetic structures under an explicit,  
deterministic, and reproducible observational regime.

The complete conceptual interpretation is presented in the book.  
This notebook documents only the corresponding experiment.

**Licence:** Creative Commons BY–NC–ND 4.0  
Reading, execution, and citation are permitted.  
Modification, derivative redistribution, or independent commercial use are not permitted.


---

## 1. The Astonishment of Observation

We have reached a point of methodological astonishment.

The previous chapters revealed a phenomenon that challenges intuition:  
prime numbers, fixed and immutable mathematical entities, exhibit **two distinct statistical natures**, depending exclusively on the way they are observed.  

* Under the **linear scale**, the spectrum of the operator follows **Poisson statistics**;
* Under the **logarithmic scale**, **GOE statistics** emerge robustly.

The inevitable question arises:  
**what is wrong?**  
Could the mathematics involved be creating an illusion — an artefact induced by the method itself?

Up to this point, no structural modification has been made to the operator.  
The only intervention introduced was the choice of the **scale of observation** — that is, the explicit introduction of an **observer**.

As in quantum physics, the system appears to respond differently depending on how it is interrogated.  
This chapter confronts this astonishment directly.

To determine whether the observed phenomenon is genuine — and not merely a reflection produced by the mathematical mirror itself — we perform the most important  
scientific test of the entire investigation: **the control experiment**.

---

## 2. The Control Experiment and an Unexpected Discovery

The central question is straightforward:

> Is the observed duality (Poisson vs GOE) a property specific to the structure of the prime numbers,
> or would any sufficiently noisy signal produce the same effect?

To answer this, we replace the deterministic arithmetic signal $\Delta_\pi(x)$ with a signal of **pure white noise** — a sequence of genuinely random numbers,  
devoid of long-range correlations.

All other elements of the protocol remain **unchanged**:

* operator construction,
* symmetrisation,
* normalisation,
* spectral analysis.

At first glance, one might expect that a signal even more “chaotic” than the primes would generate GOE statistics in both regimes.

The experimental outcome, however, reveals exactly the opposite.

---

### **The Discovery: the Chaos of the Primes Is Special**

When the code is executed with the *random signal* enabled, the result is unambiguous:

> **GOE statistics disappear completely**,
> and **both scales, linear and logarithmic, converge to Poisson**.

This result forces a profound conceptual reassessment:

1. **GOE is not generic**  
   GOE statistics do not arise from arbitrary noisy signals.  
   They require **subtle, structured long-range correlations**, which are absent in white noise.

2. **The signature of the primes is unique**  
   The emergence of GOE depends on the specific interaction between:

   * the pseudo-random structure of the primes (encoded in $\Delta_\pi(x)$);
   * the logarithmic scale, which constitutes the natural geometry of the problem.

3. **Poisson is the baseline state**  
   When the operator is driven by a perfectly decorrelated signal, its capacity to generate spectral structure collapses.  
   The outcome is inevitably a decorrelated spectrum — that is, **Poisson**.

The control experiment therefore demonstrates that GOE statistics are **not a mathematical artefact**, but an emergent and rare property, deeply tied to the  
internal structure of the prime numbers.

---

## 3. The Control Laboratory (Notebook 12)

The complete computational protocol for this control experiment is implemented in **Notebook 12**.

In it:

* the arithmetic signal $\Delta_\pi(x)$ can be switched to white noise;
* all other operator and analysis parameters are held fixed;
* the comparison between linear and logarithmic regimes is performed directly and transparently.

When toggling the control **“Use random signal?”**, one observes that:

* in the noise case, both regimes produce Poisson statistics;
* only the prime signal, when observed on the logarithmic scale, sustains the emergence of GOE.

This notebook isolates, unequivocally, signal from artefact.

---

**Operational conclusion:**

> the observed duality is not a trick of the mathematical mirror. It is a genuine property of the interaction between the prime numbers and the geometry  
> of observation.


In [1]:
# Requirements: pandas, matplotlib, numpy, ipywidgets, scipy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time

# --- Data and Matrix Generation Functions (from the previous chapter) ---
def generate_pi_data(n: int) -> np.ndarray:
    """Generate an array containing all primes up to n using an optimised sieve."""
    if n < 2:
        return np.array([], dtype=np.int64)

    size = (n - 1) // 2
    sieve = np.ones(size, dtype=bool)
    limit = int(np.sqrt(n)) // 2

    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3
            start = (p * p - 3) // 2
            sieve[start::p] = False

    indices = np.where(sieve)[0]
    odd_primes = 2 * indices + 3

    return np.concatenate((np.array([2], dtype=np.int64), odd_primes))


def get_delta_pi_for_points(x_points, primes):
    """Compute Δπ(x) for an array of x-points using a precomputed list of primes."""
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2


def generate_cos_matrix(fx_values, x_values):
    fx = fx_values.astype(np.float64)
    x = x_values.astype(np.float64)

    x[x <= 0] = 1e-12
    logx = np.log(x)

    C = np.cos(np.outer(fx, logx))
    M = C + C.T

    std_dev = M.std()
    if std_dev > 0:
        M -= M.mean()
        M /= std_dev

    return 0.5 * (M + M.T)


def local_normalize_spacings(lam, alpha=0.10, w=21):
    """
    Locally normalise level spacings (unfolding).
    This is essential for a fair comparison in potentially degenerate spectra.
    """
    N = lam.size
    k0, k1 = int(alpha * N), int((1 - alpha) * N)

    lam_bulk = np.sort(lam)[k0:k1]
    s = np.diff(lam_bulk)
    s = s[s > 0]

    if len(s) < w:
        return s / s.mean() if s.mean() > 0 else s

    w = int(w)
    if w % 2 == 0:
        w += 1

    pad = w // 2
    s_padded = np.pad(s, (pad, pad), mode='reflect')
    local_mean = np.convolve(s_padded, np.ones(w) / w, mode='valid')

    local_mean[local_mean == 0] = 1.0
    return s / local_mean


# --- Main Interactive Function (CONTROL & ROBUSTNESS) ---
def control_robustness_lab(N=2048, log_X0=8, span=2.4, use_random_signal=False):

    N_eff = N
    X0 = int(10 ** log_X0)

    # --- Data preparation ---
    if not use_random_signal:
        max_x_log = int(np.ceil(X0 * np.exp(span / 2)))
        max_x_linear = X0 + N
        max_x_needed = max(max_x_log, max_x_linear)
        pi_x_full = generate_pi_data(max_x_needed)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

    # --- Input signal generation ---
    if use_random_signal:
        print("--- USING RANDOM CONTROL SIGNAL ---")
        rng = np.random.default_rng(0)  # Fixed seed for reproducibility
        fx_linear = rng.standard_normal(N)
        fx_log = rng.standard_normal(N)
    else:
        print("--- USING PRIME SIGNAL (Δπ(x)) ---")
        fx_linear = get_delta_pi_for_points(np.arange(X0, X0 + N), pi_x_full)

        # Robust logarithmic sampling to avoid degeneracy
        x_log_raw = np.exp(
            np.linspace(np.log(X0) - span / 2, np.log(X0) + span / 2, int(N * 1.5))
        )
        x_log = np.unique(np.floor(x_log_raw)).astype(int)

        N_eff = min(N, len(x_log))
        x_log = x_log[:N_eff]
        fx_log = get_delta_pi_for_points(x_log, pi_x_full)

    # --- Linear scale analysis ---
    print("\n--- Processing Linear Scale ---")
    x_linear = np.arange(X0, X0 + N_eff)
    M_linear = generate_cos_matrix(fx_linear, x_linear)
    lam_linear, _ = np.linalg.eigh(M_linear)

    s_unfolded_linear = local_normalize_spacings(lam_linear)
    if s_unfolded_linear.size == 0:
        print("Warning: insufficient spacings (linear).")
        return

    # --- Logarithmic scale analysis ---
    print("\n--- Processing Logarithmic Scale ---")
    x_log = np.exp(
        np.linspace(np.log(X0) - span / 2, np.log(X0) + span / 2, N)
    )
    M_log = generate_cos_matrix(fx_log, x_log)
    lam_log, _ = np.linalg.eigh(M_log)

    # Use local unfolding for BOTH cases to ensure a fair comparison
    s_unfolded_log = local_normalize_spacings(lam_log)
    if s_unfolded_log.size == 0:
        print("Warning: insufficient spacings (logarithmic).")
        return

    # --- Plots ---
    s_grid = np.linspace(0, 4, 200)
    pdf_goe = (np.pi * s_grid / 2) * np.exp(-np.pi * s_grid ** 2 / 4)
    pdf_poisson = np.exp(-s_grid)

    # Left plot: Linear scale
    ax = axes[0]
    ax.hist(s_unfolded_linear, bins='auto', density=True, alpha=0.75, label='Data (Linear)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='GOE Theory')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Poisson Theory')
    ax.set_title('a) Linear Scale', fontsize=14)
    ax.set_xlabel('s (Normalised Spacing)')
    ax.set_ylabel('Density')
    ax.set_xlim(0, 4)
    ax.legend(loc='upper right')

    # Right plot: Logarithmic scale
    ax = axes[1]
    ax.hist(s_unfolded_log, bins='auto', density=True, alpha=0.75, label='Data (Log)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='GOE Theory')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Poisson Theory')
    ax.set_title('b) Logarithmic Scale', fontsize=14)
    ax.set_xlabel('s (Normalised Spacing)')
    ax.set_xlim(0, 4)
    ax.legend(loc='upper right')

    title_suffix = "RANDOM SIGNAL" if use_random_signal else f"PRIMES at X₀ ≈ {X0:g}"
    fig.suptitle(f"Robustness Analysis: {title_suffix}", fontsize=18, weight='bold')
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


# --- Interactive Widget ---
interact(
    control_robustness_lab,
    N=widgets.Dropdown(options=[512, 1024, 2048], value=2048, description='N:'),
    log_X0=widgets.IntSlider(min=5, max=8, step=1, value=8, description='X₀ = 10^', continuous_update=False),
    span=widgets.FloatSlider(min=1.0, max=4.0, step=0.1, value=2.4, description='Span (log):'),
    use_random_signal=widgets.Checkbox(value=False, description='Use random signal?')
);


interactive(children=(Dropdown(description='N:', index=2, options=(512, 1024, 2048), value=2048), IntSlider(va…

---

## 4. The Music of the Primes versus Background Noise

The control experiment implemented in this notebook finally allows a clear separation between what constitutes **genuine structure** in the prime numbers  
and what is merely **statistical noise**.

The distinction is not subtle.  
It is direct, visual, and methodologically conclusive.

---

### Comparative Summary

#### **1. The Music of the Primes**

* On the **linear scale**, the arithmetic signal of the primes produces a **decorrelated** statistic, compatible with **Poisson**.
* On the **logarithmic scale**, the very same signal produces a **strongly correlated** statistic, compatible with **GOE**.
* **Conclusion:**  
  The prime numbers exhibit two distinct statistical manifestations, dictated exclusively by the geometry of observation.

---

#### **2. Background Noise (Random Control Signal)**

* On the **linear scale**, white noise generates **Poisson** statistics.
* On the **logarithmic scale**, white noise continues to generate **Poisson** statistics.
* **Conclusion:**  
  Background noise has its own statistical signatures, but the GOE statistic is entirely absent.

---

## 5. GOE Statistics as a Structural Signature

The GOE statistic is the most complex pattern identified throughout this study.  
It represents the spectral “symphony” associated with deep, long-range correlations.

The control experiment demonstrates its origin unambiguously:

1. **GOE emerges from the primes**  
   GOE is not a generic outcome of noisy signals.  
   When the operator is driven by white noise, GOE does not appear.  
   This shows that GOE statistics are a specific property of the **internal pseudo-random structure of the prime numbers**.

2. **GOE requires geometric resonance**  
   GOE arises only when this singular arithmetic structure enters into **resonance with the logarithmic scale** — the natural scale of the prime universe.

---

## 6. Experimental Closure

The control laboratory implemented in this notebook conclusively isolates **signal** from **artefact**.

* **Poisson statistics** can be interpreted as the baseline behaviour of uncorrelated systems.
* **GOE statistics**, by contrast, are not noise. They are rare, structured, and deeply tied to the arithmetic of the primes.

> **GOE is the music of a deep and hidden order in the distribution of the prime numbers.**
